Working Hour Analysis
==

In [3]:
import psycopg2
import pandas as pd 
import sqlalchemy as salc
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import datetime
import json
warnings.filterwarnings('ignore')

with open("config.json") as config_file:
    config = json.load(config_file)

database_connection_string = 'postgres+psycopg2://{}:{}@{}:{}/{}'.format(config['user'], config['password'], config['host'], config['port'], config['database'])

dbschema='augur_data'
engine = salc.create_engine(
    database_connection_string,
    connect_args={'options': '-csearch_path={}'.format(dbschema)})

In [4]:
#declare all repo ids you would like to produce charts for
repo_set = {1} # 1 stands for augur

#can be set as 'competitors' or 'repo'
#'competitors' will group graphs by type, so it is easy to compare across repos
# 'repo' will group graphs by repo so it is easy to look at all the contributor data for each repo
display_grouping = 'repo'

#if display_grouping is set to 'competitors', enter the repo ids you do no want to alias, if 'display_grouping' is set to repo the list will not effect anything
not_aliased_repos = [1]

begin_date = '2017-10-01'
end_date = '2021-10-31'

#specify number of outliers for removal in scatter plot
scatter_plot_outliers_removed = 5
save_files = True

In [15]:
pr_all = pd.DataFrame()

for repo_id in repo_set:
    pr_query = salc.sql.text(f"""
                SELECT
                    contributors.cntrb_email as email,
                    count(pull_requests.pull_request_id) as pr_num
                    
                from
                    pull_requests,
                    contributors
                where
                    pull_requests.repo_id = {repo_id}
                    AND pull_requests.pr_augur_contributor_id = contributors.cntrb_id
                Group by contributors.cntrb_id
                Order by count(pull_requests.pull_request_id) desc
                Limit 10
                
    """)
    pr_b = pd.read_sql(pr_query, con=engine)
    if not pr_all.empty: 
        pr_all = pd.concat([pr_all, pr_b]) 
    else: 
        # first repo
        pr_all = pr_b
display(pr_all)

,email,pr_num
0,s@goggins.com,337
1,c@carterlandis.com,94
2,gabe.heim@yahoo.com,75
3,None,42
4,ma@bing0ne.com,26
5,msnell@unomaha.edu,26
6,None,22
7,saicharan.reddy1@gmail.com,20
8,None,17
9,derek@howderek.com,12
